<a href="https://colab.research.google.com/github/saddarudin/google_colab/blob/main/GenAI_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain --quiet
!pip install cohere --quiet
!pip install tiktoken --quiet
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
os.environ['COHERE_API_KEY'] = userdata.get('COHERE_API_KEY')

In [3]:
#helper function for printing docs

def pretty_printing_docs(docs):
  print(
      f"\n{'-' * 100}\n".join(
          [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
      )
  )

##Step-1 Documents Loading

In [4]:
!pip install pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 4.6 MB/s eta 0:00:00


In [5]:
from langchain.document_loaders import PyPDFLoader

In [6]:
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Agreements/EMPLOYEE_AGREEMENT.pdf

--2025-07-02 08:35:44--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Agreements/EMPLOYEE_AGREEMENT.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206079 (201K) [application/octet-stream]
Saving to: ‘EMPLOYEE_AGREEMENT.pdf’

EMPLOYEE_AGREEMENT. 100%[===================>] 201.25K  --.-KB/s    in 0.03s   

2025-07-02 08:35:44 (5.74 MB/s) - ‘EMPLOYEE_AGREEMENT.pdf’ saved [206079/206079]



In [7]:
loader = PyPDFLoader(file_path='EMPLOYEE_AGREEMENT.pdf')
pages = loader.load()
print(len(pages))

13


In [8]:
full_text = ""
for page in pages:
  full_text += page.page_content

print("Lines", len(full_text.split("\n")))
print("Words", len(full_text.split(" ")))
print("Characters", len(full_text))

Lines 369
Words 4204
Characters 29540


##Step-2 Split the data into chunks

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,chunk_overlap=30)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

133


##Step-3 Creating Embeddings

In [10]:
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(user_agent='langchain')

/tmp/ipython-input-10-3264506617.py:2: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embeddings = CohereEmbeddings(user_agent='langchain')


In [11]:
# A sample Embedding
sample_embedding = embeddings.embed_query("You must follow the rules")
print(sample_embedding)

[0.9038086, 0.22033691, -1.1132812, 0.87109375, 0.7758789, -0.8671875, -1.7265625, -3.4882812, -0.79785156, -0.9248047, 0.7158203, 0.12878418, -2.1992188, -1.4003906, 1.71875, -2.2148438, 1.2626953, 3.6875, 1.5859375, -0.8857422, 0.65478516, 0.81152344, 0.47265625, 0.39990234, 3.5410156, -0.4165039, 2.3496094, -2.7578125, -1.3652344, 0.9609375, 0.051849365, 0.72753906, 0.89208984, 1.8652344, -1.4140625, -1.7783203, 2.2910156, -0.8955078, -0.10760498, 1.2958984, -2.4003906, 1.0673828, 0.12963867, -1.7050781, 2.0976562, -1.7285156, -1.3632812, 3.1933594, 1.6357422, -0.96875, 2.4628906, 0.27783203, -0.42333984, 0.7553711, -0.7441406, -1.8066406, -3.5625, -0.84472656, -0.5419922, -0.30029297, -0.28881836, -0.79052734, 1.1015625, -1.3984375, 0.3618164, -1.7236328, -1.6503906, -1.9824219, 0.1928711, 0.72753906, 0.18774414, 1.8144531, 2.2539062, -0.56347656, 0.27514648, 1.2304688, -1.4589844, 1.5078125, 0.4831543, -1.9550781, -0.82128906, 0.85302734, -1.2285156, 1.2070312, -1.0117188, -1.3183

In [12]:
len(sample_embedding)

4096

In [13]:
sample_docs = [
    "You must follow the rules",
    "You must not disclose the rules"
]

embedded_vector1 = embeddings.embed_documents(sample_docs)
print(embedded_vector1)

[[0.9038086, 0.22033691, -1.1132812, 0.87109375, 0.7758789, -0.8671875, -1.7265625, -3.4882812, -0.79785156, -0.9248047, 0.7158203, 0.12878418, -2.1992188, -1.4003906, 1.71875, -2.2148438, 1.2626953, 3.6875, 1.5859375, -0.8857422, 0.65478516, 0.81152344, 0.47265625, 0.39990234, 3.5410156, -0.4165039, 2.3496094, -2.7578125, -1.3652344, 0.9609375, 0.051849365, 0.72753906, 0.89208984, 1.8652344, -1.4140625, -1.7783203, 2.2910156, -0.8955078, -0.10760498, 1.2958984, -2.4003906, 1.0673828, 0.12963867, -1.7050781, 2.0976562, -1.7285156, -1.3632812, 3.1933594, 1.6357422, -0.96875, 2.4628906, 0.27783203, -0.42333984, 0.7553711, -0.7441406, -1.8066406, -3.5625, -0.84472656, -0.5419922, -0.30029297, -0.28881836, -0.79052734, 1.1015625, -1.3984375, 0.3618164, -1.7236328, -1.6503906, -1.9824219, 0.1928711, 0.72753906, 0.18774414, 1.8144531, 2.2539062, -0.56347656, 0.27514648, 1.2304688, -1.4589844, 1.5078125, 0.4831543, -1.9550781, -0.82128906, 0.85302734, -1.2285156, 1.2070312, -1.0117188, -1.318

In [14]:
print(sample_embedding)
print(sample_embedding[0])

[0.9038086, 0.22033691, -1.1132812, 0.87109375, 0.7758789, -0.8671875, -1.7265625, -3.4882812, -0.79785156, -0.9248047, 0.7158203, 0.12878418, -2.1992188, -1.4003906, 1.71875, -2.2148438, 1.2626953, 3.6875, 1.5859375, -0.8857422, 0.65478516, 0.81152344, 0.47265625, 0.39990234, 3.5410156, -0.4165039, 2.3496094, -2.7578125, -1.3652344, 0.9609375, 0.051849365, 0.72753906, 0.89208984, 1.8652344, -1.4140625, -1.7783203, 2.2910156, -0.8955078, -0.10760498, 1.2958984, -2.4003906, 1.0673828, 0.12963867, -1.7050781, 2.0976562, -1.7285156, -1.3632812, 3.1933594, 1.6357422, -0.96875, 2.4628906, 0.27783203, -0.42333984, 0.7553711, -0.7441406, -1.8066406, -3.5625, -0.84472656, -0.5419922, -0.30029297, -0.28881836, -0.79052734, 1.1015625, -1.3984375, 0.3618164, -1.7236328, -1.6503906, -1.9824219, 0.1928711, 0.72753906, 0.18774414, 1.8144531, 2.2539062, -0.56347656, 0.27514648, 1.2304688, -1.4589844, 1.5078125, 0.4831543, -1.9550781, -0.82128906, 0.85302734, -1.2285156, 1.2070312, -1.0117188, -1.3183

##Step-4 Storing in Vector Stores

In [15]:
!pip install chromadb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 8.5 MB/s eta 

In [16]:
from langchain.vectorstores import Chroma

In [17]:
emp_rules_db = Chroma.from_documents(
    chunks,
    embeddings,
    persist_directory="./emp_rules_db"
    )

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


##Step-5 Retrieval

In [18]:
retriever = emp_rules_db.as_retriever()
result = retriever.get_relevant_documents("What is the policy for sick leaves?")
result

/tmp/ipython-input-18-1411391429.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = retriever.get_relevant_documents("What is the policy for sick leaves?")
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(metadata={'producer': 'Skia/PDF m123', 'creationdate': '2024-04-03T12:51:00+00:00', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '2', 'page': 1, 'total_pages': 13, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT'}, page_content='sick days and personal absence days as authorized by the Company for its other employees.\nARTICLE 6\nFringe Benefits\nEmployee shall be entitled to participate in and receive benefits from all of the Company’s employee\nbenefit plans that are now, or in the future may be, maintained by the'),
 Document(metadata={'producer': 'Skia/PDF m123', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'title': 'EMPLOYEE AGREEMENT', 'creationdate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'moddate': '2024-04-03T12:5

In [19]:
for i in range(len(result)):
  print(result[i].metadata)

{'producer': 'Skia/PDF m123', 'creationdate': '2024-04-03T12:51:00+00:00', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '2', 'page': 1, 'total_pages': 13, 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'title': 'EMPLOYEE AGREEMENT'}
{'producer': 'Skia/PDF m123', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'title': 'EMPLOYEE AGREEMENT', 'creationdate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'moddate': '2024-04-03T12:51:00+00:00', 'page_label': '2', 'page': 1, 'total_pages': 13}
{'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page_label': '2', 'title': 'EMPLOYEE AGREEMENT', 'total_pages': 13, 'source': 'EMPLOYEE_AGREEMENT.pdf', 'moddate': '2024-04-03T12:51:00+00:00', 'page': 1, 'pr

In [20]:
retriever = emp_rules_db.as_retriever(search_kwargs={"k":4})
result = retriever.get_relevant_documents("What is the policy for insurance?")
pretty_printing_docs(result)

Document 1:

Company for its employees, including, without limitation, the Company’s health insurance plan. No amounts
paid to Employee from an employee benefit plan shall count as compensation due Employee as base salary or
----------------------------------------------------------------------------------------------------
Document 2:

shareholders, officers, directors, employees, counsel, agents, affiliates and assigns (collectively, the
“Company Indemnities”) harmless from and against any and all direct or indirect demands, claims,
----------------------------------------------------------------------------------------------------
Document 3:

remedies available to the Company with respect to Employee’s breach of his obligations hereunder,
Employee shall defend, indemnify and hold the Company, the Affiliates, and each of their respective
----------------------------------------------------------------------------------------------------
Document 4:

shall promptly pay or reimburse E

In [21]:
for i in range(len(result)):
  print(result[i].metadata)

{'title': 'EMPLOYEE AGREEMENT', 'producer': 'Skia/PDF m123', 'moddate': '2024-04-03T12:51:00+00:00', 'source': 'EMPLOYEE_AGREEMENT.pdf', 'creationdate': '2024-04-03T12:51:00+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'page': 2, 'total_pages': 13, 'page_label': '3'}
{'page': 6, 'creationdate': '2024-04-03T12:51:00+00:00', 'total_pages': 13, 'source': 'EMPLOYEE_AGREEMENT.pdf', 'page_label': '7', 'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'moddate': '2024-04-03T12:51:00+00:00', 'title': 'EMPLOYEE AGREEMENT'}
{'total_pages': 13, 'moddate': '2024-04-03T12:51:00+00:00', 'creationdate': '2024-04-03T12:51:00+00:00', 'producer': 'Skia/PDF m123', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36', 'source': 'EMPLOYEE_AGREEM

In [22]:
result = retriever.get_relevant_documents("Employee monthly salary")
pretty_printing_docs(result)

Document 1:

Compensation of Employee
4.1. Base Compensation.  For all services rendered by Employee under this Employee Agreement, the
Company agrees to pay Employee the rate of $14,583 per month (the “base salary”), which shall be payable
----------------------------------------------------------------------------------------------------
Document 2:

to Employee not less frequently than bi-monthly, or as is consistent with the Company’s practice for its other
employees.  
4.2. Other Compensation.  Employee shall receive other compensation as more fully described on
Appendix B, attached hereto and made a part hereof.
----------------------------------------------------------------------------------------------------
Document 3:

Company for its employees, including, without limitation, the Company’s health insurance plan. No amounts
paid to Employee from an employee benefit plan shall count as compensation due Employee as base salary or
------------------------------------------------

##RAG Example-2

In [23]:
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Banking_System_Doc/BASEL.pdf

--2025-07-02 08:36:36--  https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/Banking_System_Doc/BASEL.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1260197 (1.2M) [application/octet-stream]
Saving to: ‘BASEL.pdf’

BASEL.pdf           100%[===================>]   1.20M  --.-KB/s    in 0.06s   

2025-07-02 08:36:36 (18.7 MB/s) - ‘BASEL.pdf’ saved [1260197/1260197]



In [24]:
# Step-1: Document Loading
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader(file_path='BASEL.pdf')
pages = loader.load()
print(len(pages))

77


In [25]:
# Step-2: Split data into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

521


In [26]:
# Step-3: Creating Embeddings
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(user_agent='langchain')

In [27]:
# Step-4: Storing in a VectorDB
basel_norms_db = Chroma.from_documents(
    chunks,
    embeddings,
    persist_directory="./basel_norms_db"
    )

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [28]:
# Step-5: Retrieval
retriever = basel_norms_db.as_retriever(search_kwargs={"k":3})
result = retriever.get_relevant_documents("What percentage is the minimum Capital Requirements?")
print(result)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[Document(metadata={'total_pages': 77, 'keywords': '', 'page_label': '68', 'agenda item1': '', 'comments': '', 'creationdate': '2011-05-31T11:52:06+02:00', 'company': '', 'meeting': '2010 - 138th Meeting of the BCBS November-December 2010', 'moddate': '2019-11-14T17:33:29+01:00', 'sort order': '0.110000000000000', 'page': 67, 'author': 'Basel Committee on Banking Supervision', 'doc #': '', 'title': 'Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version June 2011', 'description0': '', 'producer': 'Acrobat Distiller 9.4.0 (Windows)', 'creator': 'Acrobat PDFMaker 9.1 for Word', 'source': 'BASEL.pdf', 'contenttype': 'Document', 'subject': 'Full text of "Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version", June 2011'}, page_content='capital) \nMinimum Capital Conservation Ratios \n(expressed as a percentage of earnings) \n4.5% -  5.75% 100% \n>5.75% 

In [29]:
print(len(result))

3


In [30]:
print([i.metadata for i in result])

[{'total_pages': 77, 'keywords': '', 'page_label': '68', 'agenda item1': '', 'comments': '', 'creationdate': '2011-05-31T11:52:06+02:00', 'company': '', 'meeting': '2010 - 138th Meeting of the BCBS November-December 2010', 'moddate': '2019-11-14T17:33:29+01:00', 'sort order': '0.110000000000000', 'page': 67, 'author': 'Basel Committee on Banking Supervision', 'doc #': '', 'title': 'Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version June 2011', 'description0': '', 'producer': 'Acrobat Distiller 9.4.0 (Windows)', 'creator': 'Acrobat PDFMaker 9.1 for Word', 'source': 'BASEL.pdf', 'contenttype': 'Document', 'subject': 'Full text of "Basel III: A global regulatory framework for more resilient banks and banking systems - post BCBS meeting - revised version", June 2011'}, {'page_label': '64', 'author': 'Basel Committee on Banking Supervision', 'creator': 'Acrobat PDFMaker 9.1 for Word', 'agenda item1': '', 'creationdate'

##MultiQuery Retriever

In [31]:
!pip install wikipedia -q

  Preparing metadata (setup.py) ... done


In [32]:
from langchain.document_loaders import wikipedia

In [77]:
loader = wikipedia.WikipediaLoader(query="Asif Ali Zardari")
documents = loader.load()
print(len(documents))

/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


24


In [78]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=10)
chunks = text_splitter.split_documents(documents)
print(len(chunks))

995


In [79]:
from langchain.embeddings import CohereEmbeddings
embeddings = CohereEmbeddings(user_agent='langchain')

In [80]:
from langchain.vectorstores import Chroma
zardari = Chroma.from_documents(
    chunks,
    embeddings,
    persist_directory="./zardari_db"
)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [52]:
from langchain.retrievers import MultiQueryRetriever
from langchain.chains import RetrievalQA
from langchain.llms import Cohere

In [81]:
llm = Cohere(temperature=0.9)
llm_based_retriever = MultiQueryRetriever.from_llm(
    retriever=zardari.as_retriever(search_kwargs={"k":3}),
    llm=llm
)
llm_based_retriever

MultiQueryRetriever(retriever=VectorStoreRetriever(tags=['Chroma', 'CohereEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7e68be428550>, search_kwargs={'k': 3}), llm_chain=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an AI language model assistant. Your task is \n    to generate 3 different versions of the given user \n    question to retrieve relevant documents from a vector  database. \n    By generating multiple perspectives on the user question, \n    your goal is to help the user overcome some of the limitations \n    of distance-based similarity search. Provide these alternative \n    questions separated by newlines. Original question: {question}')
| Cohere(client=<cohere.client.Client object at 0x7e68b2b44910>, async_client=<cohere.client.AsyncClient object at 0x7e68d0206e10>, temperature=0.9, cohere_api_key=SecretStr('**********'))
| LineListOutputParser())

### For getting extra information about what is happening inside retriever

In [69]:
import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [82]:
question1 = "What is DOB of Asif Zardari"
question2 = "What political party does Asif Zardari belong to?"
rel_docs1 = llm_based_retriever.get_relevant_documents(question1)
rel_docs2 = llm_based_retriever.get_relevant_documents(question2)
rel_docs1

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of birth of Asif Zardari?', '2. DOB of Asif Zardari in dd/mm/yyyy format?', '3. What is the birth date of Asif Zardari, the former president of Pakistan? ', 'These variations help encapsulate the key essence of the original query while providing more context or specificity to potentially enhance search results and overcome limitations in straightforward keyword matching or numerically based searches.']
INFO:langchain.retrievers.multi_query:Generated queries: ['Here are three different variations of the query, to help with garnering information:', '1. What party was Asif Zardari affiliated with?', '2. What are the political associations of Asif Zardari?', '3. Please provide any pertinent information about the political career of Asif Zardari. ', 'These variations broadened the scope of the question by asking specifics about his affiliations, identity, and political career, in case posing the original question

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Hakim_Ali_Zardari', 'title': 'Hakim Ali Zardari', 'summary': 'Hakim Ali Zardari (9 December 1930 – 24 May 2011) was a Pakistani politician who served as a member of National Assembly of Pakistan from 1972 to 1977,1988 to 1990 and then again from 1993 to 1996.\nHe was also a landlord,a businessman and a personality associated with the film industry, both in Sindhi and in Urdu, having been a film distributor as well a film producer.'}, page_content='He was born on 9 December 1930 in the village of Fatohal Zardari to Mohammad Hussain Zardari. His'),
 Document(metadata={'title': 'Asif', 'source': 'https://en.wikipedia.org/wiki/Asif', 'summary': "Asif (Arabic: آصف) is an Arabic masculine given name. In Persian and Urdu it is often pronounced as 'Asif' or 'Asef' though the original form is 'Asaf'. This name referred to Solomon's vizier in the Islamic tradition, and by extension to a wise, prudential figure."}, page_content="Asif  A

In [83]:
rel_docs2

[Document(metadata={'title': 'Fatima Bhutto', 'source': 'https://en.wikipedia.org/wiki/Fatima_Bhutto', 'summary': "Fatima Bhutto (born 29 May 1982) is a Pakistani writer and columnist. She is the daughter of politician Murtaza Bhutto, and grand-daughter of former Pakistani prime minister and president Zulfiqar Ali Bhutto. Born in Kabul, she was raised in Syria and Karachi, and received her bachelor's degree from Barnard College in New York City, followed by a master's degree from the SOAS University of London.\nBhutto is a critic of her aunt and former prime minister Benazir Bhutto and her husband Asif Ali Zardari, whom she accused of involvement in her father's murder. Her non-fiction book Songs of Blood and Sword, released in 2010, is about her family. Bhutto has written for The News and The Guardian among other publications."}, page_content='records accounts of those affected.'),
 Document(metadata={'summary': 'The president of Pakistan (Urdu: صدرِ پاکستان, romanized: s̤adr-i Pākist

##Contextual Compression

In [58]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [84]:
llm = Cohere(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever = zardari.as_retriever(search_kwargs={'k':3})
)

In [85]:
compressed_docs = compression_retriever.get_relevant_documents(question1)

In [86]:
compressed_docs[0].metadata

{'summary': "Asif (Arabic: آصف) is an Arabic masculine given name. In Persian and Urdu it is often pronounced as 'Asif' or 'Asef' though the original form is 'Asaf'. This name referred to Solomon's vizier in the Islamic tradition, and by extension to a wise, prudential figure.",
 'source': 'https://en.wikipedia.org/wiki/Asif',
 'title': 'Asif'}

In [76]:
compressed_docs[0].metadata['summary']

"Asif (Arabic: آصف) is an Arabic masculine given name. In Persian and Urdu it is often pronounced as 'Asif' or 'Asef' though the original form is 'Asaf'. This name referred to Solomon's vizier in the Islamic tradition, and by extension to a wise, prudential figure."

## Retrieval QA Chain

In [87]:
question1 = "What is DOB of Asif Zardari?"
question = "What is the political party that Asif Zardari belongs to?"

rel_docs1 = llm_based_retriever.get_relevant_documents(question1)
rel_docs2 = llm_based_retriever.get_relevant_documents(question2)

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are 3 different versions of the question: ', '1. What is the date of birth of Asif Zardari, and his birth name? ', '2. What are the names of the parents of Asif Zardari and his birth date? ', '3. What is the full date of birth of Asif Zardari, who served as a president and has a political background? ', "These variations hope to capture the priority of understanding Asif Zardari's date of birth but with different contextual focus points. This should help in retrieving relevant documents in case the specific phrasing of his name or certain biographical details prove vague or insufficient in the database's results."]
INFO:langchain.retrievers.multi_query:Generated queries: ["Here are 3 different versions of the question, formulated to help with the retrieval of relevant documents from a vector database based on semantic similarity, not just the original question's words: ", '1. Who is Asif Zardari affiliated with politically

In [89]:
from langchain.chains import RetrievalQA

llm = Cohere(temperature=0)
QA_Chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=llm_based_retriever
)

In [90]:
query = question1
docs = QA_Chain({"query":query})
print(docs["result"])

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the date of birth of Asif Zardari?', '2. Can you provide the birth date of Asif Ali Zardari?', '3. What personal information do you have on Asif Zardari?']


 Asif Zardari was born in 1955, but I do not know the specific date of his birth. I could not find it in the provided context. 

I can promise you that Asif Zardari is not 90 years old as of today because that would mean he was born in 1934, and the provided context mentions him having a cousin who is currently (in 2022) alive and was born in 1930, meaning that Asif Zardari cannot be older than 90 years old as of today. 


In [91]:
docs

{'query': 'What is DOB of Asif Zardari?',
 'result': ' Asif Zardari was born in 1955, but I do not know the specific date of his birth. I could not find it in the provided context. \n\nI can promise you that Asif Zardari is not 90 years old as of today because that would mean he was born in 1934, and the provided context mentions him having a cousin who is currently (in 2022) alive and was born in 1930, meaning that Asif Zardari cannot be older than 90 years old as of today. '}

In [92]:
print(QA_Chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [93]:
query = question2
docs = QA_Chain({"query":query})
print(docs["result"])

INFO:langchain.retrievers.multi_query:Generated queries: ['Here are 3 different variations of your question, overly simplified, detailed, and loosely themed: ', '1. Simplified: Who is Asif Zardari politically aligned to? ', '2. Detailed: What political party did Asif Zardari spend his career contributing to, holding a leadership position, and what formative events occurred during his tenure? ', '3. Thematic: What party oversees the political affairs of governance X federally, led by Asif Zardari, who employed strategy Y as its foundation? ', "Let me know if these variations address the limitations of proximity-based search techniques you described! I'm happy to polish these if more context is needed."]


 Asif Zardari began his political career as a member of the Pakistan Peoples Party, however he has also worked in the military government of President Muhammad Zia-ul-Haq, who was affiliated with the Pakistan Muslim League (PML) and in 2018 he was elected as the chair of the Pakistan Peoples Party (PPP). Therefore Asif Zardari has been a member and leader of multiple political parties over his career. 


## Validation of the Result

In [ ]:
#COI Data
!wget https://raw.githubusercontent.com/venkatareddykonasani/Datasets/master/COI/COI.pdf
from langchain.document_loaders import PyPDFLoader

#Document Loading
loader = PyPDFLoader("COI.pdf")
pages = loader.load()

#Split the data into Chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)
chunks = text_splitter.split_documents(pages)
print(len(chunks))

#Embeddings and Vector DB
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
embeddings = OpenAIEmbeddings()

coi_db= Chroma.from_documents(chunks,
                             embeddings,
                             persist_directory="coi_db"
                             )
coi_db.persist()

#Retrieval Q_A Chain

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI , Cohere

llm=OpenAI(temperature=0)

Q_AChain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",# It takes a list of documents, inserts them all into a prompt
    retriever=coi_db.as_retriever()
)